# Autism Detection from Facial Image using TensorFlow and VGG16

Autism is a serious developmental disorder that impairs the ability to communicate and interact.

Autism spectrum disorder impacts the nervous system and affects the overall cognitive, emotional, social and physical health of the affected individual.

The range and severity of symptoms can vary widely. Common symptoms include difficulty with communication, difficulty with social interactions, obsessive interests and repetitive behaviours.
Early recognition, as well as behavioural, educational and family therapies may reduce symptoms and support development and learning.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
print(os.listdir(("/content/drive/My Drive/Colab Notebooks/Autism_Dataset")))

['Test', 'Train', 'Validate']


In [ ]:
import math
import random
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
vgg_model = tf.keras.applications.vgg16.VGG16()
print(type(vgg_model))
vgg_model.summary()

<class 'tensorflow.python.keras.engine.functional.Functional'>
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
________________________________________________________________

In [ ]:
model = keras.models.Sequential()
for layer in vgg_model.layers[0:-1]:
    model.add(layer)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [ ]:
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [ ]:
model.add(layers.Dense(1))

In [ ]:
loss = keras.losses.BinaryCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(optimizer=optim, loss=loss, metrics=metrics)

In [ ]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [ ]:
train_dir=os.getcwd()+'/drive/My Drive/Colab Notebooks/Autism_Dataset/Train'
validation_dir=os.getcwd()+'/drive/My Drive/Colab Notebooks/Autism_Dataset/Validate'

train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_batches = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    class_mode='binary',
    batch_size=100,
    shuffle=True,
    color_mode="rgb", 
)

val_batches = valid_gen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    class_mode='binary',
    batch_size=25,
    shuffle=True,
    color_mode="rgb",
)

Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [ ]:
epochs = 10

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    verbose=2
)

history = model.fit(train_batches, validation_data=val_batches,
          callbacks=[early_stopping],
          epochs=epochs, verbose=1)

Epoch 1/10
4/4 [==============================] - 250s 66s/step - loss: 0.7538 - accuracy: 0.5500 - val_loss: 0.6367 - val_accuracy: 0.6200
Epoch 2/10
4/4 [==============================] - 248s 66s/step - loss: 0.6970 - accuracy: 0.6150 - val_loss: 0.5799 - val_accuracy: 0.6300
Epoch 3/10
4/4 [==============================] - 247s 66s/step - loss: 0.6014 - accuracy: 0.6375 - val_loss: 0.5829 - val_accuracy: 0.6500
Epoch 4/10
4/4 [==============================] - 248s 66s/step - loss: 0.5351 - accuracy: 0.6950 - val_loss: 0.5771 - val_accuracy: 0.6800
Epoch 5/10
4/4 [==============================] - 248s 66s/step - loss: 0.5035 - accuracy: 0.7725 - val_loss: 0.5604 - val_accuracy: 0.7200
Epoch 6/10
4/4 [==============================] - 248s 66s/step - loss: 0.4631 - accuracy: 0.7650 - val_loss: 0.5394 - val_accuracy: 0.6800
Epoch 7/10
4/4 [==============================] - 247s 66s/step - loss: 0.4391 - accuracy: 0.7650 - val_loss: 0.5441 - val_accuracy: 0.7400
Epoch 8/10
4/4 [====

In [ ]:
test_dir=os.getcwd()+'/drive/My Drive/Colab Notebooks/Autism_Dataset/Test'
test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

test_batches = test_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    class_mode='sparse',
    batch_size=4,
    shuffle=False,
    color_mode="rgb",
)

Found 40 images belonging to 2 classes.


In [ ]:
model.evaluate(test_batches, verbose=2)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()